In [1]:
import requests
import pandas as pd
import json

In [2]:
# query open targets and return json
def gene_coloc_open_targets_query(geneid):
  """
  queries open targets for colocalization data given gene id (ENS###)

  input: geneid (ENS###)
  output: coloc dataframe with the following columns:
  phenotypeId	qtlStudyId	h3	h4	log2h4h3	study.studyId	study.traitReported	
    study.pubAuthor	leftVariant.id	tissue.id	tissue.name
  """
  # query open targets for each gene in list
  api_query = f'''query {{  
    colocalisationsForGene(geneId: "{geneid}") {{
      study{{
        studyId
        traitReported
        pubAuthor
      }}
      leftVariant {{
        id
      }}
      phenotypeId
      tissue {{
        id
        name
      }}
      qtlStudyId
      h3
      h4
      log2h4h3 
    }}
  }}'''

  #set base_url for Open Targets Genetics Portal API
  base_url = "http://genetics-api.opentargets.io/graphql"

  #perform API call and check status code of response
  r = requests.post(base_url, json={'query': api_query})
  if str(r.status_code) == '400':
    print(f'{geneid} query status code: {r.status_code}')
  
  else:
    pass

  #transform API response into JSON 
  api_response_as_json = json.loads(r.text)
  
  return api_response_as_json


def qtl_coloc_open_targets_query(variant_id):
  """
  queries open targets for qtl data given variant_id in chr_pos_a1_a2 format 
    (hg38)

  input: variant id in chr_pos_a1_a2 format (hg38)
  output: dataframe with the following columns:
  qtls gene.symbol gene.id
  """
  api_query = f'''query {{
    genesForVariant(variantId: "{variant_id}") {{
      gene {{
        id
        symbol
      }}
      qtls{{
        typeId
        sourceId
        aggregatedScore
        tissues{{
          tissue {{
            id
            name
          }}
          quantile
          beta
          pval
        }}
      }}
    }}
  }}'''

  #set base_url for Open Targets Genetics Portal API
  base_url = "http://genetics-api.opentargets.io/graphql"

  #perform API call and check status code of response
  r = requests.post(base_url, json={'query': api_query})
  if str(r.status_code) == '400':
    print(f'{variant_id} query status code: {r.status_code}')
  
  else:
    pass

  #transform API response into JSON 
  api_response_as_json = json.loads(r.text)
  
  return api_response_as_json


def phewas_open_targets_query(variant_id):
  """
  query open targets for phewas data

  input: variant id (chr_pos_a1_a2 format with hg38 positions)
  output: phewas dataframe with the following columns:
  pval	beta	nTotal	nCases	oddsRatio	eaf	se	study.studyId	
    study.traitReported	study.pubAuthor
  """
  api_query = """
    query pheWAS($variantId: String!){
      pheWAS(variantId: $variantId) {
        associations {
          study {
            studyId
            traitReported
            pubAuthor
          }
          pval
          beta
          nTotal
          nCases
          oddsRatio
          eaf
          se
        }
      }
    }""" 

  #set base_url for Open Targets Genetics Portal API
  base_url = "http://genetics-api.opentargets.io/graphql"
  #set variables object
  variables = {"variantId": variant_id}

  #perform API call and check status code of response
  r = requests.post(base_url, json={'query':api_query, "variables":variables})
  if str(r.status_code) == '400':
    print(f'{variant_id} query status code: {r.status_code}')
  
  else:
    pass

  #transform API response into JSON 
  api_response_as_json = json.loads(r.text)
  
  return api_response_as_json


def search_rsid_open_targets_query(rsid):

  """
  query open targets for snp ids in chr_pos_a1_a2 and allele freqs with rsid

  input: rsid
  output: ids and allele frequencies for different ancestry groups with columns:
  id	rsId	gnomadAFR	gnomadAMR	gnomadASJ	gnomadEAS	gnomadFIN	gnomadNFE	
    gnomadNFEEST	gnomadNFENWE	gnomadNFESEU	gnomadNFEONF	gnomadOTH

  Some information on the allele frequencies being pulled 

  "gnomAD Allele frequency (African/African-American population)"
    gnomadAFR: Float

    "gnomAD Allele frequency (Latino/Admixed American population)"
    gnomadAMR: Float

    "gnomAD Allele frequency (Ashkenazi Jewish population)"
    gnomadASJ: Float

    "gnomAD Allele frequency (East Asian population)"
    gnomadEAS: Float

    "gnomAD Allele frequency (Finnish population)"
    gnomadFIN: Float

    "gnomAD Allele frequency (Non-Finnish European population)"
    gnomadNFE: Float

    "gnomAD Allele frequency (Non-Finnish Eurpoean Estonian sub-population)"
    gnomadNFEEST: Float

    "gnomAD Allele frequency (Non-Finnish Eurpoean North-Western European sub-population)"
    gnomadNFENWE: Float

    "gnomAD Allele frequency (Non-Finnish Eurpoean Southern European sub-population)"
    gnomadNFESEU: Float

    "gnomAD Allele frequency (Non-Finnish Eurpoean Other non-Finnish European sub-population)"
    gnomadNFEONF: Float

    "gnomAD Allele frequency (Other (population not assigned) population)"
    gnomadOTH: Float
    """

  api_query = """
  query search($queryString: String!){
    search(queryString: $queryString) {
      variants{
        id
        rsId
        gnomadAFR
        gnomadAMR
        gnomadASJ
        gnomadEAS
        gnomadFIN
        gnomadNFE
        gnomadNFEEST
        gnomadNFENWE
        gnomadNFESEU
        gnomadNFEONF
        gnomadOTH
      }
    }
  }"""

  #set base_url for Open Targets Genetics Portal API
  base_url = "http://genetics-api.opentargets.io/graphql"
  #set variables object
  variables = {"queryString": rsid}

  #perform API call and check status code of response
  r = requests.post(base_url, json={'query':api_query, "variables":variables})
  if str(r.status_code) == '400':
    print(f'{rsid} query status code: {r.status_code}')
  
  else:
    pass

  #transform API response into JSON 
  api_response_as_json = json.loads(r.text)
  
  return api_response_as_json


def query_qtls(snp_list):
  """
  uses qtl_coloc_open_targets_query() function to query open targets for qtls
    given a list of snps in chr_pos_a1_a2 format (hg38)

  input: list of snps (chr_pos_a1_a2 formatted with hg38 positions)
  output: cleaned up dataframe of qtls with following columns:
  snp	gene_symbol	gene_id	type	tissue	beta	pval
  """

  total_qtls_df = pd.DataFrame()
  for snp in snp_list:
    qtl_query = qtl_coloc_open_targets_query(snp)
    qtl_query_df = pd.json_normalize(qtl_query['data']['genesForVariant'])
    if len(qtl_query_df) !=  0:
      qtl_query_df['snp'] = snp
      qtls_df = qtl_query_df.loc[qtl_query_df.qtls.map(lambda d: len(d)) > 0].reset_index(drop=True).copy()
      total_qtls_df = total_qtls_df.append(qtls_df)
  total_qtls_df = total_qtls_df.reset_index(drop=True)

  final_qtls_df = pd.DataFrame()
  for i, qtl in enumerate(total_qtls_df.qtls):
    for j, qt in enumerate(qtl):
      for k, tissue in enumerate(qt['tissues']):

        qtl_dict = {
            'snp': total_qtls_df.loc[i,'snp'],
            'gene_symbol': total_qtls_df.loc[i,'gene.symbol'],
            'gene_id': total_qtls_df.loc[i,'gene.id'],
            'type': qt['typeId'],
            'tissue': tissue['tissue']['name'],
            'beta': tissue['beta'],
            'pval': tissue['pval']
        }
        
        qtl_dict_df = pd.DataFrame.from_records([qtl_dict])
        final_qtls_df = final_qtls_df.append(qtl_dict_df).reset_index(drop=True)
        
  return final_qtls_df

In [3]:
# query coloc data given gene_id
coloc_query = gene_coloc_open_targets_query('ENSG00000160714')
coloc_query_df = pd.json_normalize(coloc_query['data']['colocalisationsForGene'])
coloc_query_df.head()

,phenotypeId,qtlStudyId,h3,h4,log2h4h3,study.studyId,study.traitReported,study.pubAuthor,leftVariant.id,tissue.id,tissue.name
0,ENSG00000160714,QUACH_2016,0.183847,0.238134,0.373264,NEALE2_2395_1,Pattern 1 | hair/balding pattern,UKB Neale v2,1_153901213_C_CA,MONOCYTE_IAV,Monocyte iav
1,ENSG00000160714,QUACH_2016,0.222941,0.116175,-0.940364,NEALE2_1050,Time spend outdoors in summer,UKB Neale v2,1_153987528_A_AAAAG,MONOCYTE_IAV,Monocyte iav
2,ENSG00000160714,QUACH_2016,0.152196,0.422189,1.471959,NEALE2_30040_raw,Mean corpuscular volume,UKB Neale v2,1_154078049_A_G,MONOCYTE_IAV,Monocyte iav
3,ENSG00000160714,QUACH_2016,0.199173,0.239907,0.268452,NEALE2_30050_raw,Mean corpuscular haemoglobin,UKB Neale v2,1_154095720_C_T,MONOCYTE_IAV,Monocyte iav
4,ENSG00000160714,QUACH_2016,0.208134,0.210694,0.017637,NEALE2_864,Number of days/week walked 10+ minutes,UKB Neale v2,1_154113167_A_T,MONOCYTE_IAV,Monocyte iav


In [4]:
# query qtls for a list of snps
# query_qtls() uses qtl_coloc_open_targets_query() for each snp id in input list and returns combined and cleaned up dataframe
test_snp_list = ['1_154453788_C_T', '1_154445939_T_C']
qtls_df = query_qtls(test_snp_list)
qtls_df.head()

,snp,gene_symbol,gene_id,type,tissue,beta,pval
0,1_154453788_C_T,UBE2Q1,ENSG00000160714,eqtl,Blood (eQTLGen),-0.047601,8.819400e-09
1,1_154453788_C_T,IL6R,ENSG00000160712,eqtl,Blood (GTEX v7),-0.106938,3.641230e-07
2,1_154453788_C_T,IL6R,ENSG00000160712,eqtl,Macrophage listeria (NEDELEC 2016),0.153303,1.443160e-06
3,1_154453788_C_T,IL6R,ENSG00000160712,eqtl,Monocyte r848 (QUACH 2016),0.119603,2.832830e-07
4,1_154453788_C_T,IL6R,ENSG00000160712,eqtl,Colon transverse (GTEX v7),-0.269436,7.835600e-06
5,1_154453788_C_T,IL6R,ENSG00000160712,eqtl,Monocyte pam3csk4 (QUACH 2016),0.119035,1.144180e-07
6,1_154453788_C_T,IL6R,ENSG00000160712,eqtl,Monocyte lps (QUACH 2016),0.116618,1.808160e-07
7,1_154453788_C_T,IL6R,ENSG00000160712,eqtl,Artery tibial (GTEX v7),-0.178372,2.417210e-09
8,1_154453788_C_T,IL6R,ENSG00000160712,eqtl,Esophagus muscularis (GTEX v7),-0.231648,1.751990e-06
9,1_154453788_C_T,IL6R,ENSG00000160712,pqtl,Blood plasma (SUN2018),1.114800,0.000000e+00


In [5]:
# query phewas data for a given snp
phewas_query = phewas_open_targets_query('1_154453788_C_T')
phewas_query_df = pd.json_normalize(phewas_query['data']['pheWAS']['associations'])
phewas_query_df.head()

,pval,beta,nTotal,nCases,oddsRatio,eaf,se,study.studyId,study.traitReported,study.pubAuthor
0,1.225010e-07,0.067727,344729,NaN,NaN,0.409706,0.012803,NEALE2_30270_raw,Mean sphered cell volume,UKB Neale v2
1,1.356400e-02,0.021674,36919,NaN,NaN,0.411766,0.008779,NEALE2_1001,Duration of strenuous sports,UKB Neale v2
2,1.837000e-04,-0.060700,40266,12194.0,0.941106,0.375928,0.016200,GCST004132,Crohn's disease,de Lange KM
3,3.510000e-02,0.106000,402099,820.0,1.111822,0.409935,0.050301,SAIGE_743_9,Osteopenia or other disorder of bone and carti...,UKB SAIGE
4,1.930000e-02,0.443000,373092,58.0,1.557372,0.410034,0.189003,SAIGE_272_12,Hyperglyceridemia,UKB SAIGE


In [6]:
# get hg38 chr_pos_a1_a2 and allele freqs for given rsid
rsid_query = search_rsid_open_targets_query("rs4129267")
rsid_query_df = pd.json_normalize(rsid_query['data']['search']['variants'])
rsid_query_df.head()

,id,rsId,gnomadAFR,gnomadAMR,gnomadASJ,gnomadEAS,gnomadFIN,gnomadNFE,gnomadNFEEST,gnomadNFENWE,gnomadNFESEU,gnomadNFEONF,gnomadOTH
0,1_154453788_C_T,rs4129267,0.137483,0.509434,0.451724,0.379534,0.301957,0.375928,0.340299,0.389706,0.528302,0.388941,0.331801
